In [14]:
import json
import base64, hashlib, hmac, time
import requests
import pandas as pd
import numpy as np
import gdax
from requests.auth import AuthBase
from websocket import create_connection
from pymongo import MongoClient
from pprint import pprint
from sklearn.metrics import mean_squared_error
import math
import time
from datetime import datetime
from statsmodels.tsa.api import VAR, DynamicVAR

/Users/Dereck/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
ws = create_connection("wss://ws-feed.gdax.com")
#ws.connect("wss://api2.bitfinex.com:3000/ws")
ws.send(json.dumps({
    "type": "subscribe",
    "channels": [{ "name": "ticker", "product_ids": ["LTC-USD"] }]
}))

while True:
    result = ws.recv()
    result = json.loads(result)
    with open('data.txt', 'a') as outfile:
        json.dump(result, outfile)
        outfile.write('\n')
        #print ("Received '%s'" % result)
# else:
#     with open('data.txt', 'a') as outfile:
#         json.dump(result, outfile)
#         outfile.writelines(result)

ws.close()

KeyboardInterrupt: 

In [10]:
def df_maker(a_list):
    ''' takes GDAX API returned list and creates a pandas df '''
    df = pd.DataFrame(a_list)
    df.columns = ['time', 'low', 'high', 'open', 'close', 'volume']
    df.time = pd.to_datetime(df.time, unit='s')
    df.index = df.time
    df = df.drop(df.columns[0], axis=1)
    df = df.iloc[::-1]
    df.drop(df.tail(1).index,inplace=True)
    return df

In [8]:
def var_forecast(data):
    model = VAR(data)
    results = model.fit(1)
    results.summary()
    lag_order = results.k_ar
    return results.forecast(data.values[-lag_order:], 1)

In [26]:
#finish_time = datetime.datetime.now() + datetime.timedelta(hours=6)
# Started this at 2:49pm 
df_hour_guess = pd.DataFrame(guess_hour, columns = ['plow', 'phigh', 'popen', 'pclose', 'pvolume'])
while True:
    guess_hour = var_forecast(df_maker(public_client.get_product_historic_rates('LTC-USD', granularity = 3600)))
    df_hour_guess.append(pd.DataFrame(guess_hour, columns = ['plow', 'phigh', 'popen', 'pclose', 'pvolume']))
    time.sleep(3600)

KeyboardInterrupt: 

In [12]:
public_client = gdax.PublicClient()

In [16]:
guess_hour = var_forecast(df_maker(public_client.get_product_historic_rates('LTC-USD', granularity = 3600)))

In [17]:
guess_hour

array([[   131.92539112,    135.26685807,    133.36708969,    133.43052157,
         17212.1212292 ]])

In [24]:
time.sleep(60) 

KeyboardInterrupt: 